In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import tensorflow.compat.v1 as tf
import tensorflow_text

In [3]:
import struct

unknown = b'\xff\xff\xff\xff'

def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    for node in graph_def.node:
        
        if node.op == 'RefSwitch':
          node.op = 'Switch'
          for index in xrange(len(node.input)):
            if 'moving_' in node.input[index]:
              node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
          node.op = 'Sub'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'AssignAdd':
          node.op = 'Add'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'Assign':
          node.op = 'Identity'
          if 'use_locking' in node.attr: del node.attr['use_locking']
          if 'validate_shape' in node.attr: del node.attr['validate_shape']
          if len(node.input) == 2:
            node.input[0] = node.input[1]
            del node.input[1]
            
        if 'Reshape/shape' in node.name or 'Reshape_1/shape' in node.name:
            b = node.attr['value'].tensor.tensor_content
            arr_int = [int.from_bytes(b[i:i + 4], 'little') for i in range(0, len(b), 4)]
            if len(arr_int):
                arr_byte = [unknown] + [struct.pack('<i', i) for i in arr_int[1:]]
                arr_byte = b''.join(arr_byte)
                node.attr['value'].tensor.tensor_content = arr_byte
            
            if len(node.attr['value'].tensor.int_val):
                node.attr['value'].tensor.int_val[0] = -1
    
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [4]:
g = load_graph('out/frozen_model.pb')

In [5]:
i = g.get_tensor_by_name('import/inputs:0')
o = g.get_tensor_by_name('import/SelectV2_3:0')
i, o

(<tf.Tensor 'import/inputs:0' shape=(None,) dtype=string>,
 <tf.Tensor 'import/SelectV2_3:0' shape=(None, 256) dtype=int32>)

In [6]:
test_sess = tf.Session(graph = g)

2022-07-07 21:46:55.075064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-07 21:46:55.081795: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-07 21:46:55.082583: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-07 21:46:55.084030: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
%%time

o_ = test_sess.run(o, feed_dict = {i: [
    'terjemah Melayu ke Inggeris: i like u',
    'terjemah Melayu ke Inggeris: hidup ini',
    'terjemah Melayu ke Inggeris: ak tak paham la.',
]})
o_.shape

2022-07-07 21:46:57.278733: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


CPU times: user 1.51 s, sys: 959 ms, total: 2.47 s
Wall time: 2.35 s


2022-07-07 21:46:57.656368: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


(3, 256)

In [8]:
import sentencepiece as spm

DEFAULT_SPM_PATH = 'sp10m.cased.ms-en.model'
sp_model = spm.SentencePieceProcessor()
sp_model.Load(DEFAULT_SPM_PATH)

True

In [9]:
for k in range(len(o_)):
    print(k, sp_model.DecodeIds(o_[k].tolist()))

0 I'm like u
1 life
2 I don't understand it.


In [10]:
with open('ms-en-right.test') as fopen:
    right = fopen.read().split('\n')
    
with open('ms-en-left.test') as fopen:
    left = fopen.read().split('\n')

In [13]:
from tqdm import tqdm

batch_size = 16

results = []
for k in tqdm(range(0, len(left), batch_size)):
    inputs = [f'terjemah Melayu ke Inggeris: {s}' for s in left[k:k + batch_size]]
    o_ = test_sess.run(o, feed_dict = {i: inputs})
    results.extend(o_.tolist())

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5475/5475 [30:40<00:00,  2.98it/s]


In [17]:
# import json

# with open('temp.json', 'w') as fopen:
#     json.dump(results, fopen)

In [ ]:
import json

with open('temp.json') as fopen:
    results = json.load(fopen)

In [13]:
len(results)

87596

In [14]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()

In [15]:
r = results[0]
sp_model.DecodeIds(r)

"The Penal Code (Malaysia) makes any sexual connection between men and women under the age of 16 as rape either committed by the woman's loyalty or not."

In [16]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    r = sp_model.DecodeIds(r)
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [17]:
refs = [filtered_right]
sys = filtered_left

In [18]:
r = bleu.corpus_score(sys, refs)

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


In [19]:
r.__dict__

{'name': 'BLEU',
 'score': 64.1407496264804,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '82.3/67.8/58.7/51.8 (BP = 0.999 ratio = 0.999 hyp_len = 1999775 ref_len = 2001100)',
 'bp': 0.9993376449144036,
 'counts': [1645886, 1297031, 1071421, 899662],
 'totals': [1999775, 1912180, 1824756, 1737752],
 'sys_len': 1999775,
 'ref_len': 2001100,
 'precisions': [82.30355915040442,
  67.82996370634564,
  58.71585022874291,
  51.77159916950175],
 'prec_str': '82.3/67.8/58.7/51.8',
 'ratio': 0.9993378641747039}